In [1]:
!pip install -U sentence-transformers
!pip install plotly==4.14.1
!pip install torch

     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 1.5MB 14.8MB/s 
     |████████████████████████████████| 1.1MB 53.2MB/s 
     |████████████████████████████████| 2.9MB 56.9MB/s 
     |████████████████████████████████| 890kB 45.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.0-cp36-none-any.whl size=102655 sha256=fd11bfbd5f1735a7ebc3a694a39381a668d77eb30cdf8eb95121080a9b8e2d8a
  Stored in directory: /root/.cache/pip/wheels/ff/76/65/50258d8b7930e909ea2f5bd006a23d520a16765af13ab45bb3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=fac24709b896617153f22c967685e96529fc18e08d3fe5f1295e27b814d307f8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
     |████████████████████████████████| 13.2MB 249kB/s 
  Found existing installation: plotly 4.4.1
    Uninstall

In [41]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [12]:
pd.set_option('max_colwidth', 800)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('using device: ', torch.cuda.get_device_name(device), flush=True)

using device:  Tesla T4


In [6]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

100%|██████████| 306M/306M [00:14<00:00, 21.4MB/s]


In [9]:
df = pd.read_csv('jokes_combined.csv', encoding='utf-8')

In [56]:
# take a sample with 2 sentences from one category and 2 from another
sample_a = df[df['category'] == 'Yo Mama'].sample(2)
sample_b = df[df['category'] == 'Political'].sample(2)
sample = pd.concat([sample_a, sample_b])

In [57]:
sample

,text,score,category
192778,Yo mama so dumb she burnt down the house using a cd burner,1.75,Yo Mama
192852,"10 Yo Mama got so fat when she jump in the air, she got stuck.9 Yo Mama is soo fat when she trip and fall she made the Grand Caryon.8 Yo Mama is soo fat when she step on the scale said ""Out of Order.""7 Yo Mama is soo fat when she wore a red rain jacket, everyone yelled ""Hey Kool-Ade!""6 Yo Mama is soo fat when she bungee jump she broke the bridge in half!5 Yo Mama is soo fat she wears a V.C.R. as a pager.4 Yo Mama is soo fat that the city gave her own zip code.3 Yo Mama is soo fat everyone at the baseball sadtium sat on her.2 Yo Mama is soo fat when she drop you off at school, she got a ticket for littering.1 Yo Mama is soo fat takes you a five mile walk around her.",1.00,Yo Mama
194391,"A man went to a doctor, and said he wanted to be able to get a job at the local Post Office, but unfortunately he was too smart. The doctor asked him his IQ, and when he gave a three-digit reply, the doctor told him that the procedure would have to involve the removal of over half of his brain. The man insisted, and since the doctor just happened to have a brand new laser device which could zap just the right portions of brain tissue, the operation was planned. The laser was hooked up to a computer which could monitor the man's declining IQ on a nice bright LED display. The doctor threw the switch and the numbers began ticking off ... 95, 94, 93, ... Suddenly the phone rang. It was the doctor's wife. They gabbed for a few minutes, the doctor forgetting completely about his patien...",1.00,Political
192728,"An engineer, an accountant, a chemist and a bureaucrat were bragging about how smart their dogs are.The engineer called to his dog, ""T-square, do your stuff"". The dog took out paper and pen, and drew a circle, a squareand a triangle. Everyone agreed he was smart.The accountant called, ""Sliderule, do your stuff"". The pooch went to the kitchen, got a dozen cookies and made four stacks of three. Everyone was impressed.The chemist called, ""Beaker, do your stuff."" The dog went to the fridge for a quart of milk, got a ten ounce glass and poured exactly eight ounces without spilling a drop. Everyone agreed that was great.The bureaucrat called, ""Coffee Break,do your stuff!"". Coffe Break ate the cookies, drank the milk, chewed the paper, claimed he injured his mouth doing so, filed a grievance ...",2.00,Political


In [58]:
sample.text.to_list()

['Yo mama so dumb she burnt down the house using a cd burner',
 '10 Yo Mama got so fat when she jump in the air, she got stuck.9 Yo Mama is soo  fat when she trip and fall she made the Grand Caryon.8 Yo Mama is soo fat when she step on the scale said "Out of Order."7 Yo Mama is soo fat when she wore a red rain jacket, everyone yelled "Hey Kool-Ade!"6 Yo Mama is soo fat when she bungee jump she broke the bridge in half!5 Yo Mama is soo fat she wears a V.C.R. as a pager.4 Yo Mama is soo fat that the city gave her own zip code.3 Yo Mama is soo fat everyone at the baseball sadtium sat on her.2 Yo Mama is soo fat when she drop you off at school, she got a ticket for littering.1 Yo Mama is soo fat takes you a five mile walk around her.',
 'A man went to a doctor, and said he wanted to be able to get a job at the local Post Office, but unfortunately he was too smart. The doctor asked him his IQ, and when he gave a three-digit reply, the doctor told him that the procedure would have to involve

In [59]:
sample_embs = model.encode(sample.text.to_list(), convert_to_tensor=True)

In [60]:
sample_texts = sample.text.to_list()

In [61]:
len(sample_embs[0])

768

In [62]:
#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(sample_embs, sample_embs)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(sample_texts[i][:30], sample_texts[j][:30], pair['score']))

A man went to a doctor, and sa 		 An engineer, an accountant, a  		 Score: 0.3656
Yo mama so dumb she burnt down 		 10 Yo Mama got so fat when she 		 Score: 0.2962
Yo mama so dumb she burnt down 		 An engineer, an accountant, a  		 Score: 0.2218
10 Yo Mama got so fat when she 		 An engineer, an accountant, a  		 Score: 0.1726
Yo mama so dumb she burnt down 		 A man went to a doctor, and sa 		 Score: 0.1292
10 Yo Mama got so fat when she 		 A man went to a doctor, and sa 		 Score: 0.0729


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204542 entries, 0 to 204541
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   text      204541 non-null  object 
 1   score     194531 non-null  float64
 2   category  13133 non-null   object 
dtypes: float64(1), object(2)
memory usage: 4.7+ MB


In [27]:
df = df.dropna(subset=['text'])

In [28]:
df.shape

(204541, 3)

In [30]:
texts = df.text.to_list()

In [31]:
%%time
embs = model.encode(texts)

Token indices sequence length is longer than the specified maximum sequence length for this model (569 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 4min 36s, sys: 2min 19s, total: 6min 56s
Wall time: 6min 56s
